In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier,LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.pipeline import Pipeline
from pyspark.sql.functions import lower, col,regexp_replace,rand
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF,Word2Vec
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
import os # chuyển os qua xài môi trường python myenv trùng với worker
os.environ["PYTHONPATH"] = "C:/Users/hongp/OneDrive/Desktop/spark/sparkenv/Lib/site-packages"

import pyspark
from pyspark.sql import SparkSession

scala_version = '2.12'  # Scala version
spark_version = '3.5.3' # Spark version

packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.0'  # Kafka version
]

PYTHON_EXECUTABLE = "C:/Users/hongp/OneDrive/Desktop/spark/sparkenv/Scripts/python.exe"

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Sentiment-Model") \
    .config("spark.jars.packages", ",".join(packages)) \
    .config("spark.pyspark.python", PYTHON_EXECUTABLE) \
    .config("spark.pyspark.driver.python", PYTHON_EXECUTABLE) \
    .config("spark.executorEnv.PYTHONPATH", PYTHON_EXECUTABLE) \
    .config("spark.executorEnv.PYSPARK_PYTHON", PYTHON_EXECUTABLE) \
    .config("spark.driver.extraClassPath", "D:/spark-3.5.3-bin-hadoop3/spark-3.5.3-bin-hadoop3/jars") \
    .config("spark.executor.extraClassPath", "D:/spark-3.5.3-bin-hadoop3/spark-3.5.3-bin-hadoop3/jars") \
    .config("spark.local.dir", "C:/sparktmp") \
    .config("spark.hadoop.io.native.lib", "false") \
    .getOrCreate()

spark


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50954)
Traceback (most recent call last):
  File "C:\Users\hongp\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\hongp\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\hongp\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\hongp\AppData\Local\Programs\Python\Python311\Lib\socketserver.py", line 755, in __init__
    self.handle()
  File "d:\spark\sparkenv\Lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "d:\spark\sparkenv\Lib\site-packages\pyspark\accumulators.py", line 267, in poll
    if self.rfi

In [3]:
from pyspark.ml import Pipeline, PipelineModel
model_rf=PipelineModel.load("models/randomforest_model")
new_data = spark.createDataFrame([Row(Comment="ngố vãi cả lồn")])
new_data.show()


+-----------------+
|          Comment|
+-----------------+
|ngố vãi cả lồn|
+-----------------+



In [4]:
model_rf.transform(new_data).select("Comment","prediction").show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "d:\spark\sparkenv\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hongp\AppData\Local\Programs\Python\Python311\Lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\spark\sparkenv\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\spark\sparkenv\Lib\site-packages\py4j\clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Erro

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [3]:
train = spark.read.csv(r"D:\spark\vihsd\train.csv", header=True, inferSchema=True)
test =spark.read.csv(r"D:\spark\vihsd\test.csv", header=True, inferSchema=True)

In [4]:
train.show(5)

+--------------------+--------+
|           free_text|label_id|
+--------------------+--------+
|Em được làm fan c...|       0|
|Đúng là bọn mắt h...|       2|
|Đậu Văn Cường giờ...|       0|
|CÔN ĐỒ CỤC SÚC VÔ...|       2|
|Từ lý thuyết đến ...|       0|
+--------------------+--------+
only showing top 5 rows



In [5]:
train_major=train.filter(train.label_id==0).orderBy(rand())
train_major=train_major.limit(5000)
train=train.filter((train.label_id==1) | (train.label_id==2)).union(train_major)

In [6]:
train = train.withColumn("label_id", train["label_id"].cast(IntegerType()))
test = test.withColumn("label_id", test["label_id"].cast(IntegerType()))

In [7]:
train.printSchema()

root
 |-- free_text: string (nullable = true)
 |-- label_id: integer (nullable = true)



In [8]:
train=train.withColumnRenamed("free_text","Comment")\
    .withColumnRenamed("label_id","Sentiment")
test=test.withColumnRenamed("free_text","Comment")\
    .withColumnRenamed("label_id","Sentiment")

In [9]:
regex_pattern = "[^a-zA-ZÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚĂĐĨŨƠàáâãèéêìíòóôõùúăđĩũơƯĂÂÊÔơưăâêô\s]"
train = train.withColumn("Comment", lower(regexp_replace(train["Comment"], regex_pattern, "")))
test = test.withColumn("Comment", lower(regexp_replace(test["Comment"], regex_pattern, "")))

In [10]:
train = train.na.drop(subset=["Comment","Sentiment"])
test=train.na.drop(subset=["Comment","Sentiment"])

In [11]:
train.select("Sentiment").distinct().show()

+---------+
|Sentiment|
+---------+
|        1|
|        2|
|        0|
+---------+



In [ ]:
tokenizer = RegexTokenizer(inputCol="Comment", outputCol="words", pattern="\\s+")

stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

word2vec = Word2Vec(vectorSize=1000, minCount=3, inputCol="filtered_words", outputCol="features")

In [13]:
precision_evaluator = MulticlassClassificationEvaluator(labelCol="Sentiment", predictionCol="prediction", metricName="precisionByLabel")
recall_evaluator = MulticlassClassificationEvaluator(labelCol="Sentiment", predictionCol="prediction", metricName="recallByLabel")
f1_evaluator = MulticlassClassificationEvaluator(labelCol="Sentiment", predictionCol="prediction", metricName="f1")

## Random Forest

In [ ]:
rf= RandomForestClassifier(featuresCol="features",labelCol="Sentiment")

pipeline_rf = Pipeline(stages=[tokenizer, stopwords_remover, word2vec, rf])

model_rf = pipeline_rf.fit(train)

In [ ]:

predictions_rf = model_rf.transform(test)
predictions_rf.select("Sentiment","prediction").distinct().show()

+---------+----------+
|Sentiment|prediction|
+---------+----------+
|        2|       1.0|
|        1|       0.0|
|        1|       2.0|
|        2|       2.0|
|        0|       2.0|
|        0|       0.0|
|        1|       1.0|
|        2|       0.0|
|        0|       1.0|
+---------+----------+



In [16]:

# Đánh giá mô hình
precision_rf = precision_evaluator.evaluate(predictions_rf)
recall_rf = recall_evaluator.evaluate(predictions_rf)
f1_rf = f1_evaluator.evaluate(predictions_rf)

print(f"Random Forest - Precision: {precision_rf:.4f}, Recall: {recall_rf:.4f}, F1-score: {f1_rf:.4f}")

Random Forest - Precision: 0.6574, Recall: 0.8942, F1-score: 0.5681


### Tuning

In [14]:
rf= RandomForestClassifier(featuresCol="features",labelCol="Sentiment")
pipeline_rf = Pipeline(stages=[tokenizer, stopwords_remover, word2vec, rf])
paramGrid = (ParamGridBuilder()
             .addGrid(rf.numTrees, [50, 100, 150])  # Số cây 
             .addGrid(rf.maxDepth, [ 10, 15])  # Độ sâu của cây 
             .build())

# Cross Validator với 3 folds
cv = CrossValidator(estimator=pipeline_rf,
                    estimatorParamMaps=paramGrid,
                    evaluator=f1_evaluator,
                    numFolds=3)  

cvModel = cv.fit(train)

predictions = cvModel.transform(test)

f1 = f1_evaluator.evaluate(predictions)
print(f"f1 (Random Forest): {f1}")

best_model = cvModel.bestModel


f1 (Random Forest): 0.9237625958393878


In [15]:
predictions.select("Sentiment","prediction").distinct().show()

+---------+----------+
|Sentiment|prediction|
+---------+----------+
|        2|       1.0|
|        1|       0.0|
|        1|       2.0|
|        2|       2.0|
|        0|       2.0|
|        0|       0.0|
|        1|       1.0|
|        2|       0.0|
|        0|       1.0|
+---------+----------+



In [16]:
# Lấy mô hình Random Forest tốt nhất từ CrossValidator
best_rf_model = cvModel.bestModel.stages[-1]  # RandomForestClassifier là stage cuối cùng trong pipeline

# In ra các tham số quan trọng
print(f"Best numTrees: {best_rf_model.getNumTrees}")
print(f"Best maxDepth: {best_rf_model.getMaxDepth()}")


Best numTrees: 100
Best maxDepth: 15


In [17]:
predictions_rf=best_model.transform(test)

In [18]:
precision_rf = precision_evaluator.evaluate(predictions_rf)
recall_rf = recall_evaluator.evaluate(predictions_rf)
f1_rf = f1_evaluator.evaluate(predictions_rf)

print(f"Random Forest - Precision: {precision_rf:.4f}, Recall: {recall_rf:.4f}, F1-score: {f1_rf:.4f}")

Random Forest - Precision: 0.9155, Recall: 0.9790, F1-score: 0.9238


In [ ]:
# best_model.save("models/randomforest_model")

## Logistic Regression

In [ ]:
lr = LogisticRegression(labelCol="Sentiment", featuresCol="features")
pipeline_lr = Pipeline(stages=[tokenizer, stopwords_remover, word2vec, lr])


model_lr = pipeline_lr.fit(train)


In [ ]:
predictions_lr = model_lr.transform(test)
predictions_lr.select("Sentiment","prediction").distinct().show()

+---------+----------+
|Sentiment|prediction|
+---------+----------+
|        2|       1.0|
|        1|       0.0|
|        1|       2.0|
|        2|       2.0|
|        0|       2.0|
|        0|       0.0|
|        1|       1.0|
|        2|       0.0|
|        0|       1.0|
+---------+----------+



In [ ]:
precision_lr = precision_evaluator.evaluate(predictions_lr)
recall_lr = recall_evaluator.evaluate(predictions_lr)
f1_lr = f1_evaluator.evaluate(predictions_lr)

print(f"Logistic Regression - Precision: {precision_lr:.4f}, Recall: {recall_lr:.4f}, F1-score: {f1_lr:.4f}")

Logistic Regression - Precision: 0.6570, Recall: 0.8904, F1-score: 0.5969


In [ ]:
new_data = spark.createDataFrame([Row(Comment="ghét vãi")])
new_data.show()

+--------+
| Comment|
+--------+
|ghét vãi|
+--------+



In [ ]:
model_lr.transform(new_data).select("Comment","prediction").show()

+--------+----------+
| Comment|prediction|
+--------+----------+
|ghét vãi|       1.0|
+--------+----------+



### Tunning

In [19]:
lr = LogisticRegression(labelCol="Sentiment", featuresCol="features")
pipeline_lr = Pipeline(stages=[tokenizer, stopwords_remover, word2vec, lr])

paramGrid = (ParamGridBuilder()
             .addGrid(lr.maxIter, [10, 50, 100])       # Tối ưu số vòng lặp
             .addGrid(lr.regParam, [0.0, 0.1, 0.01])   # Tối ưu regularization
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])  # Điều chỉnh L1 & L2
             .build())

cv = CrossValidator(estimator=pipeline_lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=f1_evaluator,
                    numFolds=3)  

cvModel = cv.fit(train)

predictions = cvModel.transform(test)

f1 = f1_evaluator.evaluate(predictions)
print(f"f1 (Logistic Regression): {f1}")

# In ra tham số tốt nhất
best_model_lr = cvModel.bestModel


f1 (Logistic Regression): 0.6101048121071495


In [25]:
# Lấy mô hình Random Forest tốt nhất từ CrossValidator
best_lr_model = cvModel.bestModel.stages[-1]  # RandomForestClassifier là stage cuối cùng trong pipeline

# In ra các tham số quan trọng
print(f"Best maxIter: {best_lr_model.getMaxIter()}")
print(f"Best regParam: {best_lr_model.getRegParam()}")
print(f"Best elasticNetParam: {best_lr_model.getElasticNetParam()}")

Best maxIter: 100
Best regParam: 0.0
Best elasticNetParam: 0.0


In [ ]:
# best_model_lr.save("models/logisticRegression_model")